In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sys
sys.path.append("..")

from plotting_tools import * 

In [ ]:
experiment_series ="pop8"
precipitation_setting = "Rain"
decay_setting = "decay"

In [ ]:
# Load the data
result_path = f"../../../preprocessing/preprocessed_data/{experiment_series}"
df = pd.read_csv(f"{result_path}/substances/{decay_setting}_{precipitation_setting}_output_scaled.csv")

In [ ]:
df["Date"] = pd.to_datetime(start_date) + pd.to_timedelta(df["time_in_minutes"], unit="min")

In [ ]:
# Filter data
covid_df = df[(df["variable"] == "COV19") & (df["manhole"].isin(manhole_names.keys()))]
covid_df["location"] = covid_df["manhole"].map(manhole_names)

# Sort locations for consistency
locations_sorted = manhole_names.values()

# Set up 4x2 subplots
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(18*cm, 12*cm), sharex=True)
axes = axes.flatten()

for i, location in enumerate(locations_sorted, start=0):  
    print(location)  
    ax = axes[i]
    ax.grid(True)

    group = covid_df.loc[covid_df["location"] == location,:].copy()
    sns.lineplot(data=group, x="Date", y="value", color=blue, ax=ax, estimator=np.mean, errorbar=("pi", 90))

    ax.set_title(location)
    if i%4 == 0:
        ax.set_ylabel("Virus levels\n[copies/l]")
    else:
        ax.set_ylabel("")
    
    
# Set x-labels for bottom row only
for ax in axes[-4:]:
    ax.set_xlabel("")
    
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# locator that places a tick on day=2 of every month
second_of_month = mdates.DayLocator(bymonthday=2)

for ax in axes:
    ax.xaxis.set_major_locator(second_of_month)
    for lbl in ax.get_xticklabels():
        lbl.set_rotation(45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))  # Format without year

# Add overall title
plt.tight_layout()
fig.savefig(f"../../plots/Supplement/concentration_{experiment_series}_{decay_setting}_{precipitation_setting}_COV19.png", dpi=300)
plt.show()
